In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, SimpleRNN
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist

# load mnist dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# compute the number of labels
num_labels = len(np.unique(y_train))

# convert to one-hot vector
#y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)

# resize and normalize
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size * image_size])
x_test = np.reshape(x_test,[-1, image_size* image_size])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(60000, 784) (60000,) (10000, 784) (10000,)


In [2]:
import numpy as np
import pickle

np.random.seed(100)
n_tasks = 5

from keras.utils import np_utils

#task_labels = [[0,1], [2,3], [4,5], [6,7], [8,9]]
task_labels = [[0,1], [4,5], [6,7], [8,9], [2,3]]
#task_labels = [[4,2], [0,6], [3,8], [9,7], [1,5],[8,9],[6,7],[5,5],[3,2],[0,1]]
#task_labels = [[16,14], [25, 7], [20, 27], [15, 1], [32, 19]]
#task_labels = [[0,9], [7,8], [3,6], [1,4], [2,5]]
#task_labels = [[9, 3], [1, 8], [7, 4], [0, 5], [6, 2]]
#task_labels = [[0,1], [2,3,1,0],[4,5,1,2], [6,7,3,0],[8,9,4,6]]
n_tasks = len(task_labels)
nb_classes  = 10
training_datasets = []
validation_datasets = []
multihead=False

for labels in task_labels:
    idx = np.in1d(y_train, labels)
    if multihead:
        label_map = np.arange(nb_classes)
        label_map[labels] = np.arange(len(labels))
        data = x_train[idx], np_utils.to_categorical(label_map[y_train[idx]], len(labels))
    else:
        data = x_train[idx], np_utils.to_categorical(y_train[idx], nb_classes)
        training_datasets.append(data)

for labels in task_labels:
    idx = np.in1d(y_test, labels)
    if multihead:
        label_map = np.arange(nb_classes)
        label_map[labels] = np.arange(len(labels))
        data = x_test[idx], np_utils.to_categorical(label_map[y_test[idx]], len(labels))
    else:
        data = x_test[idx], np_utils.to_categorical(y_test[idx], nb_classes)
        validation_datasets.append(data)
        
tasks_train={}; labels_train = {}; tasks_test = {}; labels_test = {}

for i in range(len(task_labels)):
    tasks_train[str(i)] = training_datasets[i][0]
    labels_train[str(i)] = training_datasets[i][1]
    tasks_test[str(i)] = validation_datasets[i][0]
    labels_test[str(i)] = validation_datasets[i][1]
    print('Task {0} size: Trainset - {1}, {2}, Testset - {3}, {4}'.format(i,tasks_train[str(i)].shape, labels_train[str(i)].shape, tasks_test[str(i)].shape, labels_test[str(i)].shape))

Tasks_dumped = []
for i in range(len(task_labels)):
    Tasks_dumped.append((tasks_train[str(i)], labels_train[str(i)], tasks_test[str(i)], labels_test[str(i)], tasks_test[str(i)], labels_test[str(i)]))
f = open('mnist_tasks.pkl', "wb")
pickle.dump(Tasks_dumped, f)
f.close()

Using TensorFlow backend.


Task 0 size: Trainset - (12665, 784), (12665, 10), Testset - (2115, 784), (2115, 10)
Task 1 size: Trainset - (11263, 784), (11263, 10), Testset - (1874, 784), (1874, 10)
Task 2 size: Trainset - (12183, 784), (12183, 10), Testset - (1986, 784), (1986, 10)
Task 3 size: Trainset - (11800, 784), (11800, 10), Testset - (1983, 784), (1983, 10)
Task 4 size: Trainset - (12089, 784), (12089, 10), Testset - (2042, 784), (2042, 10)
